# Edge Detection Using The Prewitt Operator
*Cody Weaver*

### What Is The Prewitt Operator
The Prewitt operator is a discrete differentiation operator that computes an approxiamation of the image intensity function gradient. The Prewitt operator can be used to detect edges in an image because the magnitude of the gradient of the image at a given pixel will have large values when the pixels nearby have large differences in intensity and smaller values when the differences in intensity are small. The gradient of the image intensity function at a given pixel is a 2D vector with $x$ and $y$ components of $G_x$ and $G_y$. The Prewitt operator uses two $3x3$ kernels.The $P_x$ kernel calculates the $x$ component of the gradient and the $P_y$ kernel calculates the $y$ component of the gradient. 

$$ P_x = \left(\begin{array}{cc} 
        1 & 0 & -1 \\
        1 & 0 & -1 \\
        1 & 0 & -1 
        \end{array}\right) $$

$\;$
        
$$ P_y = \left(\begin{array}{cc}
        1 & 1 & 1 \\
        0 & 0 & 0 \\
        -1 & -1 & -1 
        \end{array}\right) $$
        
To compute $G_x$ and $G_y$ for a given pixel you convolve the image with the kernels. To find the $x$ component for the pixel at $(a,b)$ in the image $I$ **(note that the image should be converted to grayscale before applying the Prewitt Operator)**:

$$ G_x(a,b) = \Sigma_{i=0}^2\Sigma_{j=0}^2 P_x(i,j)\;*\; I(a-1+i,\; b-1+j) $$

Likewise, finding the $y$ component is done like:

$$ G_y(a,b) = \Sigma_{i=0}^2\Sigma_{j=0}^2 P_y(i,j)\;*\; I(a-1+i,\; b-1+j) $$

You can then find the magnitude, $G_{mag}$ of the gradient vector:

$$ G_{mag} = \sqrt{G_x^2 + G_y^2} $$

The value of a pixel in the edge image is set to the value of $G_{mag}$. Pixels with a large $G_{mag}$ will be brighter than surrounding pixels and denote an edge found by the Prewitt Operator.

The Prewitt Operator can be used on a grayscale image by itself but it can be easily improved in a few ways. The first is you can only accept edge found if they are above a certain magnitude. This will help eliminate redundant edges and false positives. The second way is to blur the image first before applying the Prewitt Operator. Smoothing the grayscale image can help reduce noise and allow for cleaner edges to be found.

### Pseudocode for Implementation of Prewitt Operator
    img = readImage()
    convert img to grayscale
    optionally smooth image to reduce noise
    for i in 1, rows-2:
        for j in 1, columns-2:
            Gx = sum(sum(Px * I(i:i+2, j:j+2))
            Gy = sum(sum(Py * I(i:i+2, j:j+2))
            Gmag = sqrt(Gx^2 + Gy^2)
            if Gmag < THRESHOLD:
               Gmag = 0
            new_img[i,j] = Gmag
            
     return new_img

### The Prewitt Operator
This implementation of the Prewitt Operator can smooth and threshold the image to produce different results.

In [86]:
import cv2
import numpy as np

# Threshold values should be between 0 and 255
def prewitt_operator(img, threshold=40, smooth=True):
    rows, cols = img.shape #find image size
    edge_img = np.zeros((rows, cols)) #blank image 
    
    #P_x
    prewitt_x = np.array([[1, 0, -1], 
                          [1, 0, -1],
                          [1, 0, -1]])
    #P_y
    prewitt_y = np.array([[1, 1, 1],
                          [0, 0, 0],
                          [-1, -1, -1]])
    
    if smooth: # blurs image to reduce noise
        img = cv2.GaussianBlur(img,(3, 3),cv2.BORDER_DEFAULT)
    
    for i in range(1, rows-1): # ignores the edge pixels since G_x and G_y cannot be computed at the edge
        for j in  range(1, cols-1):
            
            G_x = ( (img[i-1,j-1] * prewitt_x[0,0]) + (img[i,j-1] * prewitt_x[0,1]) + (img[i+1,j-1] * prewitt_x[0,2])
                  + (img[i-1,j] * prewitt_x[1,0]) + (img[i,j] * prewitt_x[1, 1]) + (img[i+1,j] * prewitt_x[1,2])
                  + (img[i-1,j+1] * prewitt_x[2,0]) + (img[i,j+1] * prewitt_x[2,1]) + (img[i+1,j+1] * prewitt_x[2,2]) )
            
            G_y = ( (img[i-1,j-1] * prewitt_y[0,0]) + (img[i,j-1] * prewitt_y[0,1]) + (img[i+1,j-1] * prewitt_y[0,2])
                  + (img[i-1,j] * prewitt_y[1,0]) + (img[i,j] * prewitt_y[1,1]) + (img[i+1,j] * prewitt_y[1,2])
                  + (img[i-1,j+1] * prewitt_y[2,0]) + (img[i,j+1] * prewitt_y[2,1]) + (img[i+1,j+1] * prewitt_y[2,2]) )
            
            G_mag = np.sqrt(G_x ** 2 + G_y ** 2)
            
            if G_mag < threshold: G_mag = 0 
            
            edge_img[i, j] = G_mag
            
    return edge_img

### Before And After Prewitt Operator Without Smoothing And Thresholding

In [87]:
img = cv2.imread("img/bird.jpg", cv2.IMREAD_GRAYSCALE) # reads image and convert to grayscale
edge_img = prewitt_operator(img, threshold=0, smooth=False)
cv2.imwrite("img/bird_output.jpg", edge_img)

True

<p style="text-align: center;"><img src="img/bird.jpg"/><img src="img/bird_output.jpg"/></p> 

### Prewitt Operator With Threshold = 40 And No Smoothing

In [88]:
img = cv2.imread("img/bird.jpg", cv2.IMREAD_GRAYSCALE) # reads image and convert to grayscale
edge_img = prewitt_operator(img, threshold=40, smooth=False)
cv2.imwrite("img/bird_output_threshold.jpg", edge_img)

True

<p style="text-align: center;"><img src="img/bird.jpg"/><img src="img/bird_output_threshold.jpg"/></p> 

### Prewitt Operator With Smoothing And No Threshold

In [89]:
img = cv2.imread("img/bird.jpg", cv2.IMREAD_GRAYSCALE) # reads image and convert to grayscale
edge_img = prewitt_operator(img, threshold=0, smooth=True)
cv2.imwrite("img/bird_output_smooth.jpg", edge_img)

True

<p style="text-align: center;"><img src="img/bird.jpg"/><img src="img/bird_output_smooth.jpg"/></p> 

### Prewitt Operator With Thresholding And Smoothing


In [91]:
img = cv2.imread("img/bird.jpg", cv2.IMREAD_GRAYSCALE) # reads image and convert to grayscale
edge_img = prewitt_operator(img, 40, smooth=True)
cv2.imwrite("img/bird_output_ts.jpg", edge_img)

True

<p style="text-align: center;"><img src="img/bird.jpg"/><img src="img/bird_output_ts.jpg"/></p> 